#### Building an Excel File which includes all the UK Government Access Fuel Price Data

**This notebook has been built in Jupyter Notebook 7.4.7.**

**It using the dependencies: BeautifulSoup4, Pandas, Requests, Openpyxl**

If not already downloaded - install via terminal/cmd - or run github requirements.txt or run_script.py scripts - see README.MD in github for full directions


```
pip install pandas requests openpyxl beautifulsoup4
```


#### NOTE: The UK Government Access Fuel Price data does not cover all fuel operators or forecourts in the UK

Only those operators that have signed-up to the road fuel monitoring scheme are included. In particular, it means data for some areas may not reflect the entire price picture - this appears to be especially true for Northern Ireland, where only Asda, Sainsbury's and Tesco are listed - with only 37 sites across the entire region.

More details on the fuel access data and the monitoring scheme can be found at the following UK Government website: [Access fuel price data](https://www.gov.uk/guidance/access-fuel-price-data)

It is anticipated the access fuel price data will cease to function some time in 2026, as the data becomes a mandatory reporting system for all UK forecourt operators from February 2026, as established by [The Motor Fuel Price (Open Data) Regulations 2025](https://www.legislation.gov.uk/ukdsi/2025/9780348275308)

VE3 Global has been appointed by the Department for Energy Security and Net Zero as the official 'aggregator' who will be making the data available to registered third-parties. Guidance on third-party registration, access and usage is awaited.



In [21]:
#CODE TO BUILD EXCEL FILE in a .xlsx format containing all the data listed for fuel prices provided at the
# UK government ACCESS FUEL PRICES website: https://www.gov.uk/guidance/access-fuel-price-data

#import the modules we need to build and export the DataFrame

from bs4 import BeautifulSoup #beautifulsoup 4 package is installed as part of Anaconda environment to parse HTML
import requests 
import pandas as pd
import json
from datetime import datetime
import os  # this will allow us to specify which path we want to save the final output excel file to


#### Scraping the URLs from the UK government's Access Fuel Price Data website: 

In [22]:
# SCRAPE THE URLs:

# Define the URL of the webpage to scrape
url = 'https://www.gov.uk/guidance/access-fuel-price-data'

# use requests library to send GET request for the url

response = requests.get(url)

# Check if the request was successful - status should be 200 and parse with BeautifulSoup

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

# Initialize a list to store JSON URLs
json_url = []


# The URLs are all held within table markdown tags. To find all <td> elements in the table, using soup:

td_elements = soup.find_all('td')

# Iterate over each <td> element and extract the URLs ending with .json or .html (shell data)
for td in td_elements:
    url = td.text.strip()
    if url.startswith('https://') and url.endswith('.json') or url.endswith('.html'):
        json_url.append(url)

In [23]:
#CHECK json_url contains the various url json addresses

json_url

['https://fuelprices.asconagroup.co.uk/newfuel.json',
 'https://storelocator.asda.com/fuel_prices_data.json',
 'https://www.bp.com/en_gb/united-kingdom/home/fuelprices/fuel_prices_data.json',
 'https://fuelprices.esso.co.uk/latestdata.json',
 'https://jetlocal.co.uk/fuel_prices_data.json',
 'https://www.morrisons.com/fuel-prices/fuel.json',
 'https://moto-way.com/fuel-price/fuel_prices.json',
 'https://fuel.motorfuelgroup.com/fuel_prices_data.json',
 'https://www.rontec-servicestations.co.uk/fuel-prices/data/fuel_prices_data.json',
 'https://api.sainsburys.co.uk/v1/exports/latest/fuel_prices_data.json',
 'https://www.sgnretail.uk/files/data/SGN_daily_fuel_prices.json',
 'https://www.shell.co.uk/fuel-prices-data.html',
 'https://www.tesco.com/fuel_prices/fuel_prices_data.json']

#### Building the intitial DataFrame

We need to use the following to generate a 'header' - which gives the appearance that the request is coming from a standard web browser. Otherwise the Tesco data will reject and timeout. We will then have to run another header sequence to get the Jet Local data - and merge that with all the other data - which will return a failed o fetch error.

In [24]:

#building the dataframe

#headers - WITHOUT THIS - THE tesco_url ENTRY WILL REJECT THE REQUEST AND TIME OUT

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

#blank list to store the called json data

fuel_data = []

#loop through each URL and fetch the json data:
for url in json_url:
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        json_data = response.json()
        fuel_data.append(json_data)
    else:
        print(f"Failed to fetch data from {url}")
    
    
    
   
# Make DataFrame with pandas (pd) called df_fuel - and use the .json_normalize function get to record_path 'stations'

df_fuel = pd.json_normalize(fuel_data, record_path = 'stations')

Failed to fetch data from https://jetlocal.co.uk/fuel_prices_data.json


In [25]:
# returns the dataframe - for visual check 

df_fuel


,site_id,brand,address,postcode,location.latitude,location.longitude,prices.E10,prices.B7,prices.E5,prices.SDV
0,gcqfydrhb9q5,BP,"Anker, Weddington Road, Nuneaton.",CV10 0AD,52.52783,-1.462525,135.9,144.9,NaN,NaN
1,gcpqrk38f7ps,JET,"Ash, London Road, Wendover.",HP22 6PN,51.747124,-0.733917,131.9,145.9,NaN,NaN
2,u10nhp3xvjpq,JET,"Bassett, 215 High Road, North Weald, Epping.",CM16 6ED,51.720871,0.178065,134.7,146.7,NaN,NaN
3,gcw527se603u,BP,"Bowling Green, Scotforth Rd, Lancaster.",LA1 4SG,54.028575,-2.795245,138.9,146.9,NaN,NaN
4,gcnh7wz5tw7f,SHELL,"Bridgwater Road, Bristol.",BS41 8JP,51.409784,-2.648837,133.9,143.9,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4380,gcw1cbv06vc9,TESCO,Liverpool Road Penwortham Lancashire,PR1 9XE,53.749311,-2.728609,130.9,136.9,NaN,NaN
4381,gcpyexc91kbu,TESCO,CIRRUS B,AL7 1ZR,51.810348,-0.195639,NaN,142.9,134.9,NaN
4382,gcybn2y41ewy,TESCO,Newbottle Street Houghton Le Spring,DH4 4AU,54.848233,-1.474841,130.9,137.9,NaN,NaN
4383,gcqgheqrz210,TESCO,Station Street Atherstone,CV9 1BU,52.577777,-1.551163,130.9,137.9,139.9,NaN


### Fixing the missing Jet Local data

This is a header issue - but it's not being picked up by making the header statement above - which spoofs all the other url sites into thinking this is a normal user browser request. However - a modified headers statement can be used. We will use this to create a separate dataframe for the Jet json data - and then append to our other dataframe.

In [26]:
# Load json file from Jet URL:

url_jet = "https://jetlocal.co.uk/fuel_prices_data.json"  #The url for the jetlocal json data
headers = {"User-Agent": "Mozilla/5.0"}  # set a headers that will mimic a browser that works with this url

response = requests.get(url_jet, headers=headers)

if response.status_code == 200:
    json_data = response.json()
    df_jet = pd.DataFrame(json_data)
else:
    print(f"Request failed with status code {response.status_code}")


# place the json data in a dataframe - normalised to show the nested columns

df_jet = pd.json_normalize(json_data, record_path = 'stations')

In [27]:
# test the jet dataframe works - call df_jet

df_jet

,site_id,brand,address,postcode,location.latitude,location.longitude,prices.E5,prices.E10,prices.B7,prices.SDV
0,gcxc46cnjv5k,JET,"Eastfield Rd, South Killingholme, Immingham, N...",DN40 3DJ,53.629445,-0.251061,140.9,129.9,138.9,147.9
1,gcwv3g9d6c5u,JET,"2 Catterick Rd, Catterick Garrison, North York...",DL9 4RZ,54.379970,-1.678725,138.9,129.9,138.9,149.9
2,gcx0m777b20z,JET,"237 Warmsworth Road, Balby, Doncaster, South Y...",DN4 0TP,53.499943,-1.171062,140.9,129.9,139.9,148.9
3,gcx1zv293j7j,JET,"Bawtry Road, Selby, North Yorkshire",YO8 8NA,53.774200,-1.064940,140.9,129.9,139.9,148.9
4,gcwcuvg3dy7n,JET,"205 Dewsbury Road, Leeds Road, West Yorkshire",LS11 5HZ,53.776995,-1.544487,140.9,129.9,139.9,148.9
5,gcwys2vmydu9,JET,"Darlington Road, Coatham Mundeville, Darlingto...",DL1 3NL,54.585200,-1.563550,138.9,129.9,138.9,147.9
6,gcx1j9p97cmr,JET,"Selby Road, Askern, Doncaster, South Yorkshire",DN6 0EP,53.618997,-1.154083,140.9,129.9,139.9,148.9
7,gcqdjjeqvb68,JET,"619 Alcester Rd S, Millpool Hill, Birmingham, ...",B14 5EL,52.414185,-1.884935,138.9,129.9,139.9,148.9
8,gcwyurt7n38s,JET,"12 Chilton Way, Chilton, Ferryhill, County Durham",DL17 0SD,54.665746,-1.563566,138.9,129.9,138.9,147.9
9,gcwydxnrfzq4,JET,"Redworth Road, Redworth, Shildon, County Durham",DL4 2JT,54.619900,-1.639240,NaN,129.9,138.9,NaN


In [28]:
# append the df-jet dataframe to our df_fuel dataframe

df_combinded_fuel = pd.concat([df_fuel, df_jet], ignore_index=True)

In [29]:
#check it's worked
df_combinded_fuel

,site_id,brand,address,postcode,location.latitude,location.longitude,prices.E10,prices.B7,prices.E5,prices.SDV
0,gcqfydrhb9q5,BP,"Anker, Weddington Road, Nuneaton.",CV10 0AD,52.52783,-1.462525,135.9,144.9,NaN,NaN
1,gcpqrk38f7ps,JET,"Ash, London Road, Wendover.",HP22 6PN,51.747124,-0.733917,131.9,145.9,NaN,NaN
2,u10nhp3xvjpq,JET,"Bassett, 215 High Road, North Weald, Epping.",CM16 6ED,51.720871,0.178065,134.7,146.7,NaN,NaN
3,gcw527se603u,BP,"Bowling Green, Scotforth Rd, Lancaster.",LA1 4SG,54.028575,-2.795245,138.9,146.9,NaN,NaN
4,gcnh7wz5tw7f,SHELL,"Bridgwater Road, Bristol.",BS41 8JP,51.409784,-2.648837,133.9,143.9,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4391,gcx1j9p97cmr,JET,"Selby Road, Askern, Doncaster, South Yorkshire",DN6 0EP,53.618997,-1.154083,129.9,139.9,140.9,148.9
4392,gcqdjjeqvb68,JET,"619 Alcester Rd S, Millpool Hill, Birmingham, ...",B14 5EL,52.414185,-1.884935,129.9,139.9,138.9,148.9
4393,gcwyurt7n38s,JET,"12 Chilton Way, Chilton, Ferryhill, County Durham",DL17 0SD,54.665746,-1.563566,129.9,138.9,138.9,147.9
4394,gcwydxnrfzq4,JET,"Redworth Road, Redworth, Shildon, County Durham",DL4 2JT,54.6199,-1.63924,129.9,138.9,NaN,NaN


In [30]:
#rename df_combined back to df_fuel

df_combined = df_fuel

In [31]:
# Check how many datapoints

len(df_fuel)

4385

#### Data Validation and cleaning

If you check the dataframe for potential duplicate enteries - it will through up in excess of 100 postcodes that show more than one fuel forecourt at the same postcode. Many of these reflect fuel stations on both sides of the road, especially on major routes. However - there does appear to be a number of genuine duplications, involving the MFG and PRL brands. Every single genuine duplicated site entry for a postcode that has these two brands - and is in fact the same site. There are about 10 operating in the UK.

Knowing this we can filter them out by checking for duplicated postcodes, then removing the PRL row - IF it is in a duplicated postcode.

In [32]:
df_cleaned = df_fuel[~(df_fuel['postcode'].duplicated(keep=False) & df_fuel['address'].str.contains('PRL', case=False, na=False))]

In [33]:
df_cleaned

,site_id,brand,address,postcode,location.latitude,location.longitude,prices.E10,prices.B7,prices.E5,prices.SDV
0,gcqfydrhb9q5,BP,"Anker, Weddington Road, Nuneaton.",CV10 0AD,52.52783,-1.462525,135.9,144.9,NaN,NaN
1,gcpqrk38f7ps,JET,"Ash, London Road, Wendover.",HP22 6PN,51.747124,-0.733917,131.9,145.9,NaN,NaN
2,u10nhp3xvjpq,JET,"Bassett, 215 High Road, North Weald, Epping.",CM16 6ED,51.720871,0.178065,134.7,146.7,NaN,NaN
3,gcw527se603u,BP,"Bowling Green, Scotforth Rd, Lancaster.",LA1 4SG,54.028575,-2.795245,138.9,146.9,NaN,NaN
4,gcnh7wz5tw7f,SHELL,"Bridgwater Road, Bristol.",BS41 8JP,51.409784,-2.648837,133.9,143.9,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4380,gcw1cbv06vc9,TESCO,Liverpool Road Penwortham Lancashire,PR1 9XE,53.749311,-2.728609,130.9,136.9,NaN,NaN
4381,gcpyexc91kbu,TESCO,CIRRUS B,AL7 1ZR,51.810348,-0.195639,NaN,142.9,134.9,NaN
4382,gcybn2y41ewy,TESCO,Newbottle Street Houghton Le Spring,DH4 4AU,54.848233,-1.474841,130.9,137.9,NaN,NaN
4383,gcqgheqrz210,TESCO,Station Street Atherstone,CV9 1BU,52.577777,-1.551163,130.9,137.9,139.9,NaN


In [34]:
# rename df_fuel_clean to df_fuel_uk - use the copy method to create an independent new dataframe rather than filtered slice

df_fuel_uk = df_cleaned

#### One final bit of housekeeping - in case we want to use the Latitude and Longitdue locations later
**Rename them first - and then we want to change the figures from a string object to a real float number**

In [35]:
#rename lat and longitude columns to single words

df_fuel_lat = df_fuel_uk.rename(columns={'location.latitude' : 'latitude', 'location.longitude' : 'longitude'})

In [36]:
#convert lat and long from string object to float numbers

df_fuel_lat['latitude'] = df_fuel_lat.latitude.astype(float)
df_fuel_lat['longitude'] = df_fuel_lat.longitude.astype(float)

In [37]:
df_fuel_uk = df_fuel_lat.copy()

#### Exporting the final DataFrame to an Excel file format - in this case stored on a hard-drive volume

In [38]:
# export to file

# use datetime to append date to file name


current_date = datetime.now().strftime('%d-%m-%Y')

output = f'fuel_prices_uk_{current_date}.xlsx'

In [43]:
# Specify the path to save the file 


#Requests user to add required path location:

path_to_save = input("Please type the path location to save your file, for example C:\Data or /Volumes/portable_drive/data (DO NOT ADD FILENAME")

path_to_save_expanded = os.path.expanduser(path_to_save)

# Full file path
file_path = os.path.join(path_to_save_expanded, output)

Please type the path location to save your file, for example C:\Data or /Volumes/portable_drive/data (DO NOT ADD FILENAME /Users/herdg60/Downloads


In [42]:
#convert fuel DataFrame data to Excel format and store in directory specified above with date appended

df_fuel_uk.to_excel(file_path, index = False)

In [44]:

try:
    df_fuel_uk.to_excel(file_path, index=False)
    if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
        print(f"✅ Excel file successfully written to: {os.path.abspath(file_path)}")
    else:
        print(f"⚠️ Write completed but file not found or empty at: {file_path}")
except Exception as e:
    print(f"❌ Failed to write Excel file to '{file_path}': {e}")


✅ Excel file successfully written to: /Users/herdg60/Downloads/fuel_prices_uk_24-12-2025.xlsx
